In [ ]:
from typing import List
import warnings

import numpy as np
import geopandas as gpd
import rasterio
import pandas as pd
from shapely.geometry import LineString, Point
from shapely.errors import ShapelyDeprecationWarning
import matplotlib.pyplot as plt
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["font.family"] = "Arial"
from scipy import ndimage

from scrollstats import RASTER_PATHS
from scrollstats import BendDataset, calculate_ridge_metrics, calc_dist
from scrollstats import BendDataExtractor, TransectDataExtractor, RidgeDataExtractor

In [ ]:
bend_id = "MIS_005"

# Get rasters
dem = rasterio.open(RASTER_PATHS[bend_id]["dem"])
bin_raster = rasterio.open(RASTER_PATHS[bend_id]["bin"])

# Get vectors
bend_ds = BendDataset(bend_id)
ridges = bend_ds.get_ridges(True)
transects = bend_ds.get_transects(True)

## Extract data along transects

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", ShapelyDeprecationWarning)
    rich_transects, itx = calculate_ridge_metrics(transects, bin_raster, dem, ridges)

itx = itx.loc[bend_id]

In [ ]:
t = transects.set_index("transect_id").loc[["t_016"]]
t

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(7,4))

itx.plot(ax=ax, column="ridge_amp", markersize="ridge_width", legend=True, missing_kwds={"edgecolor":"k", "facecolor":"None"})
# itx.plot(ax=ax, markersize="ridge_width", facecolor=None)
ridges.plot(ax=ax, color="k", ls="--", lw=0.5, zorder=0)
# t.plot(ax=ax, color="k")


cbar = fig.axes[1]
min_max = itx["ridge_amp"].quantile([0,1])
cbar.set_yticks(min_max, labels = np.round(min_max, 1))

cbar.set_ylabel("Ridge Amplitude [m]", rotation=270)

ax.set_title(f"{bend_id}: Ridge width and amplitude")
ax.set_axis_off()
# plt.savefig(f"/Users/avan/FLUD/BrazosScrolls/figures/{bend_id}_HighAmplitude.png", dpi=300)

# Ridge Amplitudes
**Problem**: initial ridge amps are too high  
**Solution**: Force ridge amp calculation to only use inner measurement  

**Code Problem**: RidgeDataExtractor does not know where on the transect it currently is  
**Code Solution**: Introduce transect position to the RDE and Ridge Amp calulations  

In [ ]:
s = " False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False  True True  True  True  True  True  True  True  True  True  True  True  True True  True  True  True  True  True  True  True False False False False True  True  True  True  True  True  True  True"

s.replace(" ", "").replace("e", "e, ")

In [ ]:
a = np.array([False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, True, True, True, True, True, True, True, True])
a

What about removing the first null space from the mask makes the ridge centers of mass mismatch?

In [ ]:
chosen_transect = itx.loc["t_016"].sort_values("transect_position")
for i, row in chosen_transect.iterrows():

    geom = row["substring_geometry"]
    dem_sig = row["dem_signal_selection"]
    # bin_sig = row["bin_signal"]
    bin_sig = row["bool_mask"]
    pos = row["transect_position"]
    dq_adj = row["swale_dq_adjustment"]

    fig, ax = plt.subplots(1,1, figsize=(6, 3))
    ax.plot(dem_sig, label="DEM")
    ax.plot((bin_sig * (np.nanmax(dem_sig)- np.nanmin(dem_sig)))+ np.nanmin(dem_sig), label="Binary Classification")

    p1, p2, p3 = [Point(p) for p in geom.coords]
    d1 = p1.distance(p2) - dq_adj
    d2 = p2.distance(p3)

    x = [0, d1, d1+d2]
    y = np.ones(3)*np.nanmax(dem_sig)+1
    ax.plot(x, 
            y,
            color="k",
            marker=".",
            markersize=10,
            label = "Transect + Vertices")
    
    ax.annotate(pos, (x[1]+2, y[1]-0.1), verticalalignment="top", horizontalalignment="center", )
    ax.legend()
    ax.set_title("DEM and binary profile at the channel")
    ax.set_ylabel("Elevation [m]")
    ax.set_xlabel("Distance from channel [m]")


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3,4))

itx.plot(ax=ax, categorical=True, column="metric_confidence", markersize=10, legend=True)
ridges.plot(ax=ax, color="k", ls="--", lw=0.5, zorder=0)

# cbar = fig.axes[1]
# min_max = itx["ridge_amp"].quantile([0,1])
# cbar.set_yticks(min_max, labels = np.round(min_max, 1))

cbar.set_ylabel("Ridge Amplitude [m]", rotation=270)

ax.set_title("Confidence rating at\ntransect-ridge intersections")
ax.set_axis_off()
plt.tight_layout()
plt.savefig(f"/Users/avan/FLUD/BrazosScrolls/figures/MetricConfidence_{bend_id}.png", dpi=300)

## General Metrics I want to see

1. Variation within packets
2. Variation along transects
3. Variation along ridges
